In [2]:
import os
import pandas as pd
import polars as pl


In [5]:
# Instalacja wymaganych bibliotek do pracy z Kaggle API
!pip install kaggle


In [6]:
from google.colab import files
files.upload()  # Kliknij przycisk "Choose Files" i wybierz pobrany plik 'kaggle.json'


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"patrykklytta","key":"af68a20172587b2995eec7fd5d78524e"}'}

In [7]:
# Ustawienie odpowiedniego katalogu i przeniesienie pliku kaggle.json
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/root/.kaggle"  # Ustawienie folderu na plik kaggle.json

# Przenieśmy nasz plik JSON do folderu .kaggle, który zostanie automatycznie użyty przez API
!mkdir -p /root/.kaggle
!mv kaggle.json /root/.kaggle/


In [8]:
# Pobieranie danych z Kaggle
!kaggle competitions download -c jane-street-real-time-market-data-forecasting


100% 11.4G/11.5G [01:11<00:00, 256MB/s]
100% 11.5G/11.5G [01:12<00:00, 171MB/s]


In [9]:
# Rozpakowanie danych (jeśli są w formacie zip)
!unzip jane-street-real-time-market-data-forecasting.zip


Archive:  jane-street-real-time-market-data-forecasting.zip
  inflating: features.csv            
  inflating: kaggle_evaluation/__init__.py  
  inflating: kaggle_evaluation/core/__init__.py  
  inflating: kaggle_evaluation/core/base_gateway.py  
  inflating: kaggle_evaluation/core/generated/__init__.py  
  inflating: kaggle_evaluation/core/generated/kaggle_evaluation_pb2.py  
  inflating: kaggle_evaluation/core/generated/kaggle_evaluation_pb2_grpc.py  
  inflating: kaggle_evaluation/core/kaggle_evaluation.proto  
  inflating: kaggle_evaluation/core/relay.py  
  inflating: kaggle_evaluation/core/templates.py  
  inflating: kaggle_evaluation/jane_street_gateway.py  
  inflating: kaggle_evaluation/jane_street_inference_server.py  
  inflating: lags.parquet/date_id=0/part-0.parquet  
  inflating: responders.csv          
  inflating: sample_submission.csv   
  inflating: test.parquet/date_id=0/part-0.parquet  
  inflating: train.parquet/partition_id=0/part-0.parquet  
  inflating: train.p

In [9]:
import psutil

# Sprawdzenie dostępnej pamięci
memory = psutil.virtual_memory()
print(f"Całkowita pamięć: {memory.total / (1024 ** 3)} GB")
print(f"Dostępna pamięć: {memory.available / (1024 ** 3)} GB")


Całkowita pamięć: 12.674762725830078 GB
Dostępna pamięć: 11.318534851074219 GB


In [1]:
!pip install dask


In [3]:
import os
import pandas as pd
import polars as pl
import dask.dataframe as dd


train_dir = '/content/train.parquet'

# Listujemy pliki w folderze 'train.parquet' aby zobaczyć, jak wygląda struktura
files = [os.path.join(root, name)
         for root, dirs, files in os.walk(train_dir)
         for name in files if name.endswith('.parquet')]

print(f"Znaleziono {len(files)} plików Parquet")

# Wczytanie danych przy pomocy Dask (wczytanie z różnych plików)
train_df = dd.read_parquet(files)

# Możemy teraz sprawdzić kilka pierwszych wierszy
train_df.head()


Znaleziono 10 plików Parquet


,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id
0,0,0,1,3.889038,NaN,NaN,NaN,NaN,NaN,0.851033,...,0.738489,-0.069556,1.380875,2.005353,0.186018,1.218368,0.775981,0.346999,0.095504,0
1,0,0,7,1.370613,NaN,NaN,NaN,NaN,NaN,0.676961,...,2.965889,1.190077,-0.523998,3.849921,2.626981,5.000000,0.703665,0.216683,0.778639,0
2,0,0,9,2.285698,NaN,NaN,NaN,NaN,NaN,1.056285,...,-0.864488,-0.280303,-0.326697,0.375781,1.271291,0.099793,2.109352,0.670881,0.772828,0
3,0,0,10,0.690606,NaN,NaN,NaN,NaN,NaN,1.139366,...,0.408499,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516,0
4,0,0,14,0.440570,NaN,NaN,NaN,NaN,NaN,0.955200,...,-0.373387,-0.502764,-0.348021,-3.928148,-1.591366,-5.000000,-3.572820,-1.089123,-5.000000,0


In [3]:
import os
import glob
import pandas as pd
import polars as pl

# Załadowanie pliku testowego (test.parquet)
test_files = glob.glob('test.parquet/*/*')
test_df = pd.concat([pd.read_parquet(file) for file in test_files], ignore_index=True)
print(test_df.head())


   row_id  date_id  time_id  symbol_id    weight  is_scored  feature_00  \
0       0        0        0          0  3.169998       True         0.0   
1       1        0        0          1  2.165993       True         0.0   
2       2        0        0          2  3.065550       True         0.0   
3       3        0        0          3  2.698642       True         0.0   
4       4        0        0          4  1.803330       True         0.0   

   feature_01  feature_02  feature_03  ...  feature_69  feature_70  \
0         0.0         0.0         0.0  ...        -0.0        -0.0   
1        -0.0         0.0         0.0  ...        -0.0        -0.0   
2        -0.0         0.0         0.0  ...         0.0        -0.0   
3         0.0         0.0         0.0  ...         0.0        -0.0   
4        -0.0         0.0         0.0  ...        -0.0        -0.0   

   feature_71  feature_72  feature_73  feature_74  feature_75  feature_76  \
0         0.0         0.0         NaN         NaN  

In [4]:
# Załadowanie pliku lagów (lags.parquet)
lags_files = glob.glob('lags.parquet/*/*')
lags_df = pd.concat([pd.read_parquet(file) for file in lags_files], ignore_index=True)
print(lags_df.head())


   date_id  time_id  symbol_id  responder_0_lag_1  responder_1_lag_1  \
0        0        0          0          -0.442215          -0.322407   
1        0        0          1          -0.651829          -1.707840   
2        0        0          2          -0.656373          -0.264575   
3        0        0          3          -0.188186          -0.190970   
4        0        0          4          -0.257462          -0.471325   

   responder_2_lag_1  responder_3_lag_1  responder_4_lag_1  responder_5_lag_1  \
0           0.143594          -0.926890          -0.782236          -0.036595   
1          -0.893942          -1.065488          -1.871338          -0.615652   
2          -0.892879          -1.511886          -1.033480          -0.378265   
3          -0.701490           0.098453          -1.015506          -0.054984   
4          -0.297420           0.074018          -0.324194          -0.597093   

   responder_6_lag_1  responder_7_lag_1  responder_8_lag_1  
0          -1.30574

In [5]:
# Załadowanie features.csv
features_df = pd.read_csv('features.csv')
print(features_df.head())

# Załadowanie responders.csv
responders_df = pd.read_csv('responders.csv')
print(responders_df.head())


      feature  tag_0  tag_1  tag_2  tag_3  tag_4  tag_5  tag_6  tag_7  tag_8  \
0  feature_00  False  False   True  False  False  False  False  False  False   
1  feature_01  False  False   True  False  False  False  False  False  False   
2  feature_02  False  False   True  False  False  False  False  False  False   
3  feature_03  False  False   True  False  False  False  False  False  False   
4  feature_04  False  False   True  False  False  False  False  False  False   

   tag_9  tag_10  tag_11  tag_12  tag_13  tag_14  tag_15  tag_16  
0  False   False   False   False   False    True   False    True  
1  False   False   False   False    True    True   False    True  
2  False   False   False    True   False   False   False    True  
3  False   False   False   False    True   False   False    True  
4  False   False   False    True    True   False   False    True  
     responder  tag_0  tag_1  tag_2  tag_3  tag_4
0  responder_0   True  False   True  False  False
1  responder_1   

In [ ]:
#przykład z Kaggle!
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )